In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import utility_functions
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200) # So we can see all columns
pd.set_option('display.max_rows',200)

In [11]:
import importlib
importlib.reload(utility_functions)

<module 'utility_functions' from 'd:\\Banque Misr Internship\\loan-defaulter-prediction\\Final Notebooks\\utility_functions.py'>

Read the new file with reduced features

In [12]:
df = pd.read_csv('D:/Banque Misr Internship/Loan Datasets/sampled_data_1.csv')

In [13]:
df.head(5)

SK_ID_CURR  TARGET CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN   
0      100002       1           M            N               Y             0  \
1      100003       0           F            N               N             0   
2      100004       0           M            Y               Y             0   
3      100006       0           F            N               Y             0   
4      100007       0           M            N               Y             0   

   AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE NAME_TYPE_SUITE   
0          202500.0    406597.5      24700.5         351000.0   Unaccompanied  \
1          270000.0   1293502.5      35698.5        1129500.0          Family   
2           67500.0    135000.0       6750.0         135000.0   Unaccompanied   
3          135000.0    312682.5      29686.5         297000.0   Unaccompanied   
4          121500.0    513000.0      21865.5         513000.0   Unaccompanied   

  NAME_INCOME_TYPE            NAME_EDUCATION_TYPE    NAME_FAMILY_STATUS   
0          Working  Secondary / secondary special  Single / not married  \
1    State servant               Higher education               Married   
2          Working  Secondary / secondary special  Single / not married   
3          Working  Secondary / secondary special        Civil marriage   
4          Working  Secondary / secondary special  Single / not married   

   NAME_HOUSING_TYPE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED   
0  House / apartment                    0.018801       -9461           -637  \
1  House / apartment                    0.003541      -16765          -1188   
2  House / apartment                    0.010032      -19046           -225   
3  House / apartment                    0.008019      -19005          -3039   
4  House / apartment                    0.028663      -19932          -3038   

  OCCUPATION_TYPE       ORGANIZATION_TYPE  EXT_SOURCE_2  EXT_SOURCE_3   
0        Laborers  Business Entity Type 3      0.262949      0.139376  \
1      Core staff                  School      0.622246           NaN   
2        Laborers              Government      0.555912      0.729567   
3        Laborers  Business Entity Type 3      0.650442           NaN   
4      Core staff                Religion      0.322738           NaN   

   OBS_30_CNT_SOCIAL_CIRCLE  DEF_30_CNT_SOCIAL_CIRCLE   
0                       2.0                       2.0  \
1                       1.0                       0.0   
2                       0.0                       0.0   
3                       2.0                       0.0   
4                       0.0                       0.0   

   OBS_60_CNT_SOCIAL_CIRCLE  DEF_60_CNT_SOCIAL_CIRCLE  DAYS_LAST_PHONE_CHANGE   
0                       2.0                       2.0                 -1134.0  \
1                       1.0                       0.0                  -828.0   
2                       0.0                       0.0                  -815.0   
3                       2.0                       0.0                  -617.0   
4                       0.0                       0.0                 -1106.0   

   FLAG_DOCUMENT_3  AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_YEAR  
0                1                         0.0                         1.0  
1                1                         0.0                         0.0  
2                0                         0.0                         0.0  
3                1                         NaN                         NaN  
4                0                         0.0                         0.0

We will first convert the negative values to positive using absolute

In [14]:
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].abs()
df['DAYS_BIRTH'] = df['DAYS_BIRTH'].abs()

Convert any XNA value to NaN to be removed later

In [15]:
df_cleaned = df.replace('XNA', pd.NA)

In [16]:
df_cleaned.head()


SK_ID_CURR  TARGET CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN   
0      100002       1           M            N               Y             0  \
1      100003       0           F            N               N             0   
2      100004       0           M            Y               Y             0   
3      100006       0           F            N               Y             0   
4      100007       0           M            N               Y             0   

   AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE NAME_TYPE_SUITE   
0          202500.0    406597.5      24700.5         351000.0   Unaccompanied  \
1          270000.0   1293502.5      35698.5        1129500.0          Family   
2           67500.0    135000.0       6750.0         135000.0   Unaccompanied   
3          135000.0    312682.5      29686.5         297000.0   Unaccompanied   
4          121500.0    513000.0      21865.5         513000.0   Unaccompanied   

  NAME_INCOME_TYPE            NAME_EDUCATION_TYPE    NAME_FAMILY_STATUS   
0          Working  Secondary / secondary special  Single / not married  \
1    State servant               Higher education               Married   
2          Working  Secondary / secondary special  Single / not married   
3          Working  Secondary / secondary special        Civil marriage   
4          Working  Secondary / secondary special  Single / not married   

   NAME_HOUSING_TYPE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED   
0  House / apartment                    0.018801        9461            637  \
1  House / apartment                    0.003541       16765           1188   
2  House / apartment                    0.010032       19046            225   
3  House / apartment                    0.008019       19005           3039   
4  House / apartment                    0.028663       19932           3038   

  OCCUPATION_TYPE       ORGANIZATION_TYPE  EXT_SOURCE_2  EXT_SOURCE_3   
0        Laborers  Business Entity Type 3      0.262949      0.139376  \
1      Core staff                  School      0.622246           NaN   
2        Laborers              Government      0.555912      0.729567   
3        Laborers  Business Entity Type 3      0.650442           NaN   
4      Core staff                Religion      0.322738           NaN   

   OBS_30_CNT_SOCIAL_CIRCLE  DEF_30_CNT_SOCIAL_CIRCLE   
0                       2.0                       2.0  \
1                       1.0                       0.0   
2                       0.0                       0.0   
3                       2.0                       0.0   
4                       0.0                       0.0   

   OBS_60_CNT_SOCIAL_CIRCLE  DEF_60_CNT_SOCIAL_CIRCLE  DAYS_LAST_PHONE_CHANGE   
0                       2.0                       2.0                 -1134.0  \
1                       1.0                       0.0                  -828.0   
2                       0.0                       0.0                  -815.0   
3                       2.0                       0.0                  -617.0   
4                       0.0                       0.0                 -1106.0   

   FLAG_DOCUMENT_3  AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_YEAR  
0                1                         0.0                         1.0  
1                1                         0.0                         0.0  
2                0                         0.0                         0.0  
3                1                         NaN                         NaN  
4                0                         0.0                         0.0

Retrieve Categorical features


In [17]:
categorical_features = []
categorical_features = utility_functions.get_categorical_features(df_cleaned)
print("Categorical Features:",categorical_features )

Categorical Features: ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE']


We will now one-hot encode the categorical features

In [18]:

# One-hot encode the categorical columns
df_encoded = pd.get_dummies(df_cleaned, columns=categorical_features, drop_first=False)

# Check the resulting DataFrame
df_encoded.head()

SK_ID_CURR  TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT   
0      100002       1             0          202500.0    406597.5  \
1      100003       0             0          270000.0   1293502.5   
2      100004       0             0           67500.0    135000.0   
3      100006       0             0          135000.0    312682.5   
4      100007       0             0          121500.0    513000.0   

   AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH   
0      24700.5         351000.0                    0.018801        9461  \
1      35698.5        1129500.0                    0.003541       16765   
2       6750.0         135000.0                    0.010032       19046   
3      29686.5         297000.0                    0.008019       19005   
4      21865.5         513000.0                    0.028663       19932   

   DAYS_EMPLOYED  EXT_SOURCE_2  EXT_SOURCE_3  OBS_30_CNT_SOCIAL_CIRCLE   
0            637      0.262949      0.139376                       2.0  \
1           1188      0.622246           NaN                       1.0   
2            225      0.555912      0.729567                       0.0   
3           3039      0.650442           NaN                       2.0   
4           3038      0.322738           NaN                       0.0   

   DEF_30_CNT_SOCIAL_CIRCLE  OBS_60_CNT_SOCIAL_CIRCLE   
0                       2.0                       2.0  \
1                       0.0                       1.0   
2                       0.0                       0.0   
3                       0.0                       2.0   
4                       0.0                       0.0   

   DEF_60_CNT_SOCIAL_CIRCLE  DAYS_LAST_PHONE_CHANGE  FLAG_DOCUMENT_3   
0                       2.0                 -1134.0                1  \
1                       0.0                  -828.0                1   
2                       0.0                  -815.0                0   
3                       0.0                  -617.0                1   
4                       0.0                 -1106.0                0   

   AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_YEAR  CODE_GENDER_F   
0                         0.0                         1.0          False  \
1                         0.0                         0.0           True   
2                         0.0                         0.0          False   
3                         NaN                         NaN           True   
4                         0.0                         0.0          False   

   CODE_GENDER_M  FLAG_OWN_CAR_N  FLAG_OWN_CAR_Y  FLAG_OWN_REALTY_N   
0           True            True           False              False  \
1          False            True           False               True   
2           True           False            True              False   
3          False            True           False              False   
4           True            True           False              False   

   FLAG_OWN_REALTY_Y  NAME_TYPE_SUITE_Children  NAME_TYPE_SUITE_Family   
0               True                     False                   False  \
1              False                     False                    True   
2               True                     False                   False   
3               True                     False                   False   
4               True                     False                   False   

   NAME_TYPE_SUITE_Group of people  NAME_TYPE_SUITE_Other_A   
0                            False                    False  \
1                            False                    False   
2                            False                    False   
3                            False                    False   
4                            False                    False   

   NAME_TYPE_SUITE_Other_B  NAME_TYPE_SUITE_Spouse, partner   
0                    False                            False  \
1                    False                            False   
2                  

Check for nulls in any of the columns

In [19]:
utility_functions.get_numerical_summary(df_encoded)

AMT_ANNUITY : 12 (0.004%)
AMT_GOODS_PRICE : 278 (0.09%)
EXT_SOURCE_2 : 660 (0.215%)
EXT_SOURCE_3 : 60965 (19.825%)
OBS_30_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
DEF_30_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
OBS_60_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
DEF_60_CNT_SOCIAL_CIRCLE : 1021 (0.332%)
DAYS_LAST_PHONE_CHANGE : 1 (0.0%)
AMT_REQ_CREDIT_BUREAU_HOUR : 41519 (13.502%)
AMT_REQ_CREDIT_BUREAU_YEAR : 41519 (13.502%)


{'AMT_ANNUITY': 0.003902299429939092,
 'AMT_GOODS_PRICE': 0.09040327012692229,
 'EXT_SOURCE_2': 0.21462646864665005,
 'EXT_SOURCE_3': 19.825307062186393,
 'OBS_30_CNT_SOCIAL_CIRCLE': 0.3320206431639844,
 'DEF_30_CNT_SOCIAL_CIRCLE': 0.3320206431639844,
 'OBS_60_CNT_SOCIAL_CIRCLE': 0.3320206431639844,
 'DEF_60_CNT_SOCIAL_CIRCLE': 0.3320206431639844,
 'DAYS_LAST_PHONE_CHANGE': 0.00032519161916159093,
 'AMT_REQ_CREDIT_BUREAU_HOUR': 13.501630835970097,
 'AMT_REQ_CREDIT_BUREAU_YEAR': 13.501630835970097}

Drop the remaining null rows (They won't affect the dataset that much)

In [20]:
final_df = df_encoded.dropna()


In [21]:
final_df

SK_ID_CURR  TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT   
0           100002       1             0          202500.0    406597.5  \
2           100004       0             0           67500.0    135000.0   
5           100008       0             0           99000.0    490495.5   
6           100009       0             1          171000.0   1560726.0   
7           100010       0             0          360000.0   1530000.0   
...            ...     ...           ...               ...         ...   
307503      456247       0             0          112500.0    345510.0   
307505      456249       0             0          112500.0    225000.0   
307508      456253       0             0          153000.0    677664.0   
307509      456254       1             0          171000.0    370107.0   
307510      456255       0             0          157500.0    675000.0   

        AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH   
0           24700.5         351000.0                    0.018801        9461  \
2            6750.0         135000.0                    0.010032       19046   
5           27517.5         454500.0                    0.035792       16941   
6           41301.0        1395000.0                    0.035792       13778   
7           42075.0        1530000.0                    0.003122       18850   
...             ...              ...                         ...         ...   
307503      17770.5         247500.0                    0.022800       11870   
307505      22050.0         225000.0                    0.022800       24384   
307508      29979.0         585000.0                    0.005002       14966   
307509      20205.0         319500.0                    0.005313       11961   
307510      49117.5         675000.0                    0.046220       16856   

        DAYS_EMPLOYED  EXT_SOURCE_2  EXT_SOURCE_3  OBS_30_CNT_SOCIAL_CIRCLE   
0                 637      0.262949      0.139376                       2.0  \
2                 225      0.555912      0.729567                       0.0   
5                1588      0.354225      0.621226                       0.0   
6                3130      0.724000      0.492060                       1.0   
7                 449      0.714279      0.540654                       2.0   
...               ...           ...           ...                       ...   
307503            399      0.501221      0.609276                       9.0   
307505         365243      0.346391      0.742182                       0.0   
307508           7921      0.535722      0.218859                       6.0   
307509           4786      0.514163      0.661024                       0.0   
307510           1262      0.708569      0.113922                       0.0   

        DEF_30_CNT_SOCIAL_CIRCLE  OBS_60_CNT_SOCIAL_CIRCLE   
0                            2.0                       2.0  \
2                            0.0                       0.0   
5                            0.0                       0.0   
6                            0.0                       1.0   
7                            0.0                       2.0   
...                          ...                       ...   
307503                       3.0                       9.0   
307505                       0.0                       0.0   
307508                       0.0                       6.0   
307509                       0.0                       0.0   
307510                       0.0                       0.0   

        DEF_60_CNT_SOCIAL_CIRCLE  DAYS_LAST_PHONE_CHANGE  FLAG_DOCUMENT_3   
0                            2.0                 -1134.0                1  \
2                            0.0                  -815.0                0   
5                            0.0                 -2536.0                1   
6                            0.0                 -1562.0                0   
7                            0.0                 -1070.0                1   
..

Save the final dataframe as a csv for use by the three models

In [22]:
filepath = Path('D:/Banque Misr Internship/Loan Datasets/final_dataset.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_df.to_csv(filepath)